In [ ]:
from google.cloud import storage, bigquery, pubsub_v1
# import matplotlib.pyplot as plt
import pandas as pd
import csv
import requests
from datetime import datetime
from io import StringIO
import time

In [ ]:
# Clean data in bus_stops.csv --> Remove duplicates
# Upload file to GCS
# Create an External Table pointing to the file locates in GCS so I dont have to updated it manually

# --- UPDATE PROCESS ---

# Fecth Naptan Ids that were already captured
# API Arrivals/Bus and get all Naptan Ids
# FilterOut the NaptanIds that already Exist
# If NO new Naptan Ids , Stop the program
# Ie new Naptan Ids, A StopPoint Call with remaining NaptanIds to fetch Latitude and Longitude
# Store in GCS Path gs://your-bucket-name/naptan_data/year=2025/month=07/day=09/hour=14/naptan_snapshot.csv

# Maybe to delete : bus_stop_points/naptan_data/year=2025/month=07/day=12/hour=13/minute=57


In [ ]:
# collected_naptanIds = pd.read_csv('/Users/delgadonoriega/Desktop/gcp-data-eng-bootcamp/Module_5_streaming_project/london_transport_api/bus_stops.csv')
# collected_naptanIds= collected_naptanIds[['naptanId', 'commonName',	'latitude', 'longitud']]
# collected_naptanIds.head()

,naptanId,commonName,latitude,longitud
0,490000115M,Hornchurch Station,51.55469,0.21965
1,490001243Y,Romford Station,51.57360,0.18393
2,490G00003049,Abbs Cross Lane,51.56309,0.21461
3,490G00003258,Alma Avenue,51.55465,0.22979
4,490G00004001,Billet Lane,51.56313,0.21654


In [ ]:
# unique_collected_naptanIds = collected_naptanIds.drop_duplicates(subset='naptanId', keep=False)
# unique_collected_naptanIds.to_csv('/Users/delgadonoriega/Desktop/gcp-data-eng-bootcamp/Module_5_streaming_project/london_transport_api/bus_stops.csv', index=False)


In [59]:

# unique_collected_naptanIds.plot(kind="scatter", x="latitude", y="longitud", color="purple", title="London Bus stops")

In [60]:
# Store existing NaptanIds in Cloud Storage

# PROJECT_ID = 'lon-trans-streaming-pipeline'
# BUCKET_NAME = 'bus_stop_points'
# now = datetime.now()
# DESTINATION_PATH =  f"naptan_data/year={now.year}/month={now.month:02}/day={now.day:02}/hour={now.hour:02}/minute={now.minute:02}/naptan_snapshot.csv"

# client = storage.Client(project=PROJECT_ID)
# bucket = client.bucket(BUCKET_NAME)
# blob = bucket.blob(DESTINATION_PATH)

# csv_buffer = StringIO()
# unique_collected_naptanIds.to_csv(csv_buffer, index=False)

# blob.upload_from_string(csv_buffer.getvalue(), content_type="text/csv")

In [ ]:
# Read Table and fetch unique naptan Ids already captured
PROJECT_ID = 'lon-trans-streaming-pipeline'

BQ_client = bigquery.Client(project=PROJECT_ID)
data_set = 'bus_density_streaming_pipeline'
table = 'stopspoint_coordinates'

# SQL query — adjust table name
query = f"""
    SELECT DISTINCT naptanId
    FROM `{PROJECT_ID}.{data_set}.{table}`
"""
query_job = BQ_client.query(query)
results = query_job.result()

existingNaptanIds = [row["naptanId"] for row in results]
print(len(existingNaptanIds))
existingNaptanIds

19200


['490004235N',
 '490011899N',
 '490014180N',
 '490003921S',
 '490009978S',
 '490004769W',
 '490013342E',
 '490008356E',
 '1590003501',
 '490008950S',
 '490000515Z',
 '490005733J',
 '490020190N',
 '490005425M',
 '490001027BB',
 '490007713D',
 '490011961N',
 '490015177L',
 '490004181W',
 '490014056E',
 '490012629S',
 '490005148H',
 '490004232N',
 '490013805E',
 '490005115E',
 '490016360S',
 '490003646E',
 '490005092N1',
 '490007613S',
 '490006061W',
 '490007915S',
 '490000132A',
 '490009310W',
 '490013049W',
 '490006492N',
 '490006292S',
 '490014417W',
 '490008928E',
 '490018846N',
 '40004408239A',
 '490004689N',
 '490010587S',
 '490007992N',
 '490002387ZY',
 '490002277ZZ',
 '490018674Z',
 '490009740E',
 '490013512V',
 '490005861N2',
 '490013143S',
 '490011927E',
 '490004374W',
 '490003154S',
 '490003417N',
 '490008934W',
 '490013865N',
 '490009818E',
 '490015079E',
 '490005029S',
 '490011989W1',
 '210021903896',
 '490014585N',
 '490016149W',
 '490004705W',
 '490011361G',
 '490013438E',


In [62]:
# API Call of buss arrivals

arrivals_url = "https://api.tfl.gov.uk/Mode/bus/Arrivals"
app_key = '132c49c6367b496ba654bc8092f0610a'
url_append = f'?app_key={app_key}' 
print(arrivals_url + url_append)
arrivals_response = requests.get(arrivals_url + url_append)

if arrivals_response:
    arrivals_response_json = arrivals_response.json()
    naptanids = list(set([record['naptanId'] for record in arrivals_response_json]))

https://api.tfl.gov.uk/Mode/bus/Arrivals?app_key=132c49c6367b496ba654bc8092f0610a


In [63]:
naptan_to_capture = [n for n in naptanids if n not in existingNaptanIds]
print(len(naptan_to_capture))
naptan_to_capture[0:10]

8557


['490003550XY',
 '490003781N',
 '490003412E',
 '490004432E',
 '490016327S',
 '490008844N',
 '490006184E',
 '40004410093A',
 '490001013D',
 '490004653N']

In [ ]:
# StopPoint API CALL

api_errors = [] 
iteration = 0
time_sleep_iterator = 0 
batch_result = [('naptanId', 'commonName', 'latitude', 'longitud')]


# GCS cconfiguration
BUCKET_NAME = 'bus_stop_points'

# Pub/Sub configuration
TOPIC_ID = 'naptan_ids_recieved_trigger_again'
publisher = pubsub_v1.PublisherClient()
topic_path = publisher.topic_path(PROJECT_ID, TOPIC_ID)
message_str = f"NaptanId Collection Processes finished, run again"
message_bytes = message_str.encode("utf-8")

if naptan_to_capture:

    for naptan_id in naptan_to_capture[0:100]: # Will process some at a time not all, eventutally data will be smaller

        # API CALL FOR COORDINATES MAPPING
        station_map_url = f'https://api.tfl.gov.uk/StopPoint/{naptan_id}'
        res = requests.get(station_map_url)

        if res.status_code == 200:
            station = res.json()
            commonName = station['commonName']
            latitude =  station['lat']
            longitud = station['lon']
            batch_result.append((naptan_id,commonName,latitude,longitud))
            print(f'Iteration {iteration} ✅')
            iteration += 1
            
            if time_sleep_iterator == 5:
                time.sleep(5)
                time_sleep_iterator = 0
            else:
                time_sleep_iterator += 1
        
        else:
            print(f'Some errors in iteration {iteration} found, NaptanId: {naptan_id}, status code: {res.status_code}')
            api_errors.append(naptan_id)
            iteration += 1
            continue

    # Store in GCS bucket
    now = datetime.now()
    DESTINATION_PATH =  f"naptan_data/year={now.year}/month={now.month:02}/day={now.day:02}/hour={now.hour:02}/minute={now.minute:02}/naptan_snapshot.csv"
    client = storage.Client(project=PROJECT_ID)
    bucket = client.bucket(BUCKET_NAME)
    blob = bucket.blob(DESTINATION_PATH)

    csv_buffer = StringIO()
    writer = csv.writer(csv_buffer)
    writer.writerows(batch_result)

    blob.upload_from_string(csv_buffer.getvalue(), content_type="text/csv")

    print(f'Stored {len(batch_result)-1} in GCS and now sending a message to pubsub to trigger process again')

    time.sleep(30)
    # Send message tu pub/sub    
    future = publisher.publish(topic_path, message_bytes)
    
else:
    print('No new Naptan Ids to capture')
    time.sleep(30)
    future = publisher.publish(topic_path, message_bytes)

Iteration 0 ✅
Iteration 1 ✅
Iteration 2 ✅
Iteration 3 ✅
Iteration 4 ✅
Iteration 5 ✅
Iteration 6 ✅
Iteration 7 ✅
Iteration 8 ✅
Iteration 9 ✅
Iteration 10 ✅
Iteration 11 ✅
Iteration 12 ✅
Iteration 13 ✅
Iteration 14 ✅
Iteration 15 ✅
Iteration 16 ✅
Iteration 17 ✅
Iteration 18 ✅
Iteration 19 ✅
Iteration 20 ✅
Iteration 21 ✅
Iteration 22 ✅
Iteration 23 ✅
Iteration 24 ✅
Iteration 25 ✅
Iteration 26 ✅
Iteration 27 ✅
Iteration 28 ✅
Iteration 29 ✅
Iteration 30 ✅
Iteration 31 ✅
Iteration 32 ✅
Iteration 33 ✅
Iteration 34 ✅
Iteration 35 ✅
Iteration 36 ✅
Iteration 37 ✅
Iteration 38 ✅
Iteration 39 ✅
Iteration 40 ✅
Iteration 41 ✅
Iteration 42 ✅
Iteration 43 ✅
Iteration 44 ✅
Iteration 45 ✅
Iteration 46 ✅
Iteration 47 ✅
Iteration 48 ✅
Iteration 49 ✅
Iteration 50 ✅
Iteration 51 ✅
Iteration 52 ✅
Iteration 53 ✅
Iteration 54 ✅
Iteration 55 ✅
Iteration 56 ✅
Iteration 57 ✅
Iteration 58 ✅
Iteration 59 ✅
Iteration 60 ✅
Iteration 61 ✅
Iteration 62 ✅
Iteration 63 ✅
Iteration 64 ✅
Iteration 65 ✅
Iteration 66 ✅
Itera

In [ ]:
import streamlit as st
import pydeck as pdk
import pandas as pd

# Read your enriched CSV or query from BQ
df = pd.read_csv("/Users/delgadonoriega/Desktop/gcp-data-eng-bootcamp/Module_5_streaming_project/london_transport_api/bus_stops.csv")
df = df [['commonName','latitude', 'longitude']]
df
# df.rename({'latitude': 'LAT','longitud':'LON'})
st.title("🚌 Real-time Bus Density Map")
st.map(df)  


2025-07-13 00:06:30.791 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-13 00:06:30.792 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-13 00:06:30.792 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-13 00:06:30.823 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-13 00:06:30.823 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-13 00:06:30.824 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()